In [1]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news3_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

D:\anacoda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [16]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.5,5.5,0.5):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  80
iter:  0  ,docID  2000
len(Topics)  93
iter:  0  ,docID  3000
len(Topics)  111
iter:  0  ,docID  4000
len(Topics)  124
iter:  0  ,docID  5000
len(Topics)  131
iter:  0  ,docID  6000
len(Topics)  146
iter:  0  ,docID  7000
len(Topics)  150
iter:  0  ,docID  8000
len(Topics)  165


KeyboardInterrupt: 

In [ ]:
print(alpha_NMI)
print(alpha_topics)

In [12]:
print(alpha_NMI)
print(alpha_topics)

{0.5: 0.8612696877895757, 1.0: 0.8614742856525397, 1.5: 0.8696799712535451, 2.0: 0.8483676884764597}
{0.5: 325, 1.0: 373, 1.5: 406, 2.0: 513}


In [17]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  75
iter:  0  ,docID  2000
len(Topics)  89
iter:  0  ,docID  3000
len(Topics)  101
iter:  0  ,docID  4000
len(Topics)  115
iter:  0  ,docID  5000
len(Topics)  119
iter:  0  ,docID  6000
len(Topics)  128
iter:  0  ,docID  7000
len(Topics)  129
iter:  0  ,docID  8000
len(Topics)  145
iter:  0  ,docID  9000
len(Topics)  149
iter:  0  ,docID  10000
len(Topics)  152
iter:  0  ,docID  11000
len(Topics)  152
iter:  0  ,docID  12000
len(Topics)  157
iter:  0  ,docID  13000
len(Topics)  158
iter:  0  ,docID  14000
len(Topics)  163
iter:  0  ,docID  15000
len(Topics)  165
iter:  0  ,docID  16000
len(Topics)  168
iter:  0  ,docID  17000
len(Topics)  168
iter:  0  ,docID  18000
len(Topics)  170
iter:  0  ,docID  19000
len(Topics)  171
iter:  0  ,docID  20000
len(Topics)  173
iter:  0  ,docID  21000
len(Topics)  175
iter:  0  ,docID  22000
len(Topics)  177
iter:  0  ,docID  23000
len(Topics)  177
iter:  0  ,docID  24000
len(Topic

iter:  6  ,docID  9000
len(Topics)  240
iter:  6  ,docID  10000
len(Topics)  240
iter:  6  ,docID  11000
len(Topics)  240
iter:  6  ,docID  12000
len(Topics)  240
iter:  6  ,docID  13000
len(Topics)  240
iter:  6  ,docID  14000
len(Topics)  239
iter:  6  ,docID  15000
len(Topics)  239
iter:  6  ,docID  16000
len(Topics)  239
iter:  6  ,docID  17000
len(Topics)  240
iter:  6  ,docID  18000
len(Topics)  240
iter:  6  ,docID  19000
len(Topics)  240
iter:  6  ,docID  20000
len(Topics)  242
iter:  6  ,docID  21000
len(Topics)  242
iter:  6  ,docID  22000
len(Topics)  242
iter:  6  ,docID  23000
len(Topics)  242
iter:  6  ,docID  24000
len(Topics)  242
iter:  6  ,docID  25000
len(Topics)  242
iter:  6  ,docID  26000
len(Topics)  243
iter:  6  ,docID  27000
len(Topics)  244
iter:  6  ,docID  28000
len(Topics)  244
iter:  6  ,docID  29000
len(Topics)  244
iter:  6  ,docID  30000
len(Topics)  244
alpha  0.1 Topics  244  iter  6   0.8476084665209211
iter:  7  ,docID  0
len(Topics)  244
iter:  7 

iter:  2  ,docID  18000
len(Topics)  245
iter:  2  ,docID  19000
len(Topics)  245
iter:  2  ,docID  20000
len(Topics)  243
iter:  2  ,docID  21000
len(Topics)  245
iter:  2  ,docID  22000
len(Topics)  245
iter:  2  ,docID  23000
len(Topics)  245
iter:  2  ,docID  24000
len(Topics)  244
iter:  2  ,docID  25000
len(Topics)  244
iter:  2  ,docID  26000
len(Topics)  245
iter:  2  ,docID  27000
len(Topics)  246
iter:  2  ,docID  28000
len(Topics)  248
iter:  2  ,docID  29000
len(Topics)  249
iter:  2  ,docID  30000
len(Topics)  250
alpha  0.2 Topics  250  iter  2   0.8293293275700118
iter:  3  ,docID  0
len(Topics)  250
iter:  3  ,docID  1000
len(Topics)  251
iter:  3  ,docID  2000
len(Topics)  250
iter:  3  ,docID  3000
len(Topics)  251
iter:  3  ,docID  4000
len(Topics)  254
iter:  3  ,docID  5000
len(Topics)  254
iter:  3  ,docID  6000
len(Topics)  256
iter:  3  ,docID  7000
len(Topics)  255
iter:  3  ,docID  8000
len(Topics)  257
iter:  3  ,docID  9000
len(Topics)  258
iter:  3  ,docID 

iter:  8  ,docID  26000
len(Topics)  310
iter:  8  ,docID  27000
len(Topics)  309
iter:  8  ,docID  28000
len(Topics)  310
iter:  8  ,docID  29000
len(Topics)  308
iter:  8  ,docID  30000
len(Topics)  307
alpha  0.2 Topics  307  iter  8   0.8541406592633735
iter:  9  ,docID  0
len(Topics)  307
iter:  9  ,docID  1000
len(Topics)  307
iter:  9  ,docID  2000
len(Topics)  308
iter:  9  ,docID  3000
len(Topics)  308
iter:  9  ,docID  4000
len(Topics)  307
iter:  9  ,docID  5000
len(Topics)  306
iter:  9  ,docID  6000
len(Topics)  305
iter:  9  ,docID  7000
len(Topics)  305
iter:  9  ,docID  8000
len(Topics)  303
iter:  9  ,docID  9000
len(Topics)  302
iter:  9  ,docID  10000
len(Topics)  302
iter:  9  ,docID  11000
len(Topics)  302
iter:  9  ,docID  12000
len(Topics)  302
iter:  9  ,docID  13000
len(Topics)  301
iter:  9  ,docID  14000
len(Topics)  300
iter:  9  ,docID  15000
len(Topics)  299
iter:  9  ,docID  16000
len(Topics)  298
iter:  9  ,docID  17000
len(Topics)  298
iter:  9  ,docID 

iter:  5  ,docID  2000
len(Topics)  286
iter:  5  ,docID  3000
len(Topics)  288
iter:  5  ,docID  4000
len(Topics)  289
iter:  5  ,docID  5000
len(Topics)  289
iter:  5  ,docID  6000
len(Topics)  288
iter:  5  ,docID  7000
len(Topics)  290
iter:  5  ,docID  8000
len(Topics)  291
iter:  5  ,docID  9000
len(Topics)  291
iter:  5  ,docID  10000
len(Topics)  293
iter:  5  ,docID  11000
len(Topics)  294
iter:  5  ,docID  12000
len(Topics)  296
iter:  5  ,docID  13000
len(Topics)  296
iter:  5  ,docID  14000
len(Topics)  297
iter:  5  ,docID  15000
len(Topics)  296
iter:  5  ,docID  16000
len(Topics)  294
iter:  5  ,docID  17000
len(Topics)  293
iter:  5  ,docID  18000
len(Topics)  294
iter:  5  ,docID  19000
len(Topics)  294
iter:  5  ,docID  20000
len(Topics)  294
iter:  5  ,docID  21000
len(Topics)  294
iter:  5  ,docID  22000
len(Topics)  294
iter:  5  ,docID  23000
len(Topics)  295
iter:  5  ,docID  24000
len(Topics)  295
iter:  5  ,docID  25000
len(Topics)  295
iter:  5  ,docID  26000


...

iter:  8  ,docID  30000
len(Topics)  368
alpha  0.7 Topics  368  iter  8   0.8611088535481697
iter:  9  ,docID  0
len(Topics)  368
iter:  9  ,docID  1000
len(Topics)  370
iter:  9  ,docID  2000
len(Topics)  370
iter:  9  ,docID  3000
len(Topics)  373
iter:  9  ,docID  4000
len(Topics)  376
iter:  9  ,docID  5000
len(Topics)  376
iter:  9  ,docID  6000
len(Topics)  374
iter:  9  ,docID  7000
len(Topics)  375
iter:  9  ,docID  8000
len(Topics)  373
iter:  9  ,docID  9000
len(Topics)  373
iter:  9  ,docID  10000
len(Topics)  372
iter:  9  ,docID  11000
len(Topics)  371
iter:  9  ,docID  12000
len(Topics)  371
iter:  9  ,docID  13000
len(Topics)  370
iter:  9  ,docID  14000
len(Topics)  373
iter:  9  ,docID  15000
len(Topics)  371
iter:  9  ,docID  16000
len(Topics)  369
iter:  9  ,docID  17000
len(Topics)  368
iter:  9  ,docID  18000
len(Topics)  368
iter:  9  ,docID  19000
len(Topics)  368
iter:  9  ,docID  20000
len(Topics)  364
iter:  9  ,docID  21000
len(Topics)  362
iter:  9  ,docID 

iter:  5  ,docID  6000
len(Topics)  384
iter:  5  ,docID  7000
len(Topics)  383
iter:  5  ,docID  8000
len(Topics)  380
iter:  5  ,docID  9000
len(Topics)  381
iter:  5  ,docID  10000
len(Topics)  382
iter:  5  ,docID  11000
len(Topics)  381
iter:  5  ,docID  12000
len(Topics)  382
iter:  5  ,docID  13000
len(Topics)  383
iter:  5  ,docID  14000
len(Topics)  385
iter:  5  ,docID  15000
len(Topics)  387
iter:  5  ,docID  16000
len(Topics)  397
iter:  5  ,docID  17000
len(Topics)  401
iter:  5  ,docID  18000
len(Topics)  401
iter:  5  ,docID  19000
len(Topics)  403
iter:  5  ,docID  20000
len(Topics)  405
iter:  5  ,docID  21000
len(Topics)  404
iter:  5  ,docID  22000
len(Topics)  404
iter:  5  ,docID  23000
len(Topics)  403
iter:  5  ,docID  24000
len(Topics)  403
iter:  5  ,docID  25000
len(Topics)  403
iter:  5  ,docID  26000
len(Topics)  403
iter:  5  ,docID  27000
len(Topics)  405
iter:  5  ,docID  28000
len(Topics)  407
iter:  5  ,docID  29000
len(Topics)  405
iter:  5  ,docID  30

iter:  1  ,docID  15000
len(Topics)  292
iter:  1  ,docID  16000
len(Topics)  302
iter:  1  ,docID  17000
len(Topics)  312
iter:  1  ,docID  18000
len(Topics)  312
iter:  1  ,docID  19000
len(Topics)  313
iter:  1  ,docID  20000
len(Topics)  315
iter:  1  ,docID  21000
len(Topics)  316
iter:  1  ,docID  22000
len(Topics)  318
iter:  1  ,docID  23000
len(Topics)  319
iter:  1  ,docID  24000
len(Topics)  319
iter:  1  ,docID  25000
len(Topics)  320
iter:  1  ,docID  26000
len(Topics)  321
iter:  1  ,docID  27000
len(Topics)  323
iter:  1  ,docID  28000
len(Topics)  326
iter:  1  ,docID  29000
len(Topics)  328
iter:  1  ,docID  30000
len(Topics)  329
alpha  0.9 Topics  330  iter  1   0.8245717529290658
iter:  2  ,docID  0
len(Topics)  330
iter:  2  ,docID  1000
len(Topics)  335
iter:  2  ,docID  2000
len(Topics)  336
iter:  2  ,docID  3000
len(Topics)  337
iter:  2  ,docID  4000
len(Topics)  336
iter:  2  ,docID  5000
len(Topics)  338
iter:  2  ,docID  6000
len(Topics)  340
iter:  2  ,doc

iter:  7  ,docID  23000
len(Topics)  465
iter:  7  ,docID  24000
len(Topics)  465
iter:  7  ,docID  25000
len(Topics)  465
iter:  7  ,docID  26000
len(Topics)  467
iter:  7  ,docID  27000
len(Topics)  465
iter:  7  ,docID  28000
len(Topics)  464
iter:  7  ,docID  29000
len(Topics)  468
iter:  7  ,docID  30000
len(Topics)  467
alpha  0.9 Topics  466  iter  7   0.8527718444476899
iter:  8  ,docID  0
len(Topics)  466
iter:  8  ,docID  1000
len(Topics)  464
iter:  8  ,docID  2000
len(Topics)  465
iter:  8  ,docID  3000
len(Topics)  462
iter:  8  ,docID  4000
len(Topics)  461
iter:  8  ,docID  5000
len(Topics)  457
iter:  8  ,docID  6000
len(Topics)  456
iter:  8  ,docID  7000
len(Topics)  457
iter:  8  ,docID  8000
len(Topics)  458
iter:  8  ,docID  9000
len(Topics)  459
iter:  8  ,docID  10000
len(Topics)  458
iter:  8  ,docID  11000
len(Topics)  458
iter:  8  ,docID  12000
len(Topics)  459
iter:  8  ,docID  13000
len(Topics)  458
iter:  8  ,docID  14000
len(Topics)  461
iter:  8  ,docID 

iter:  4  ,docID  1000
len(Topics)  387
iter:  4  ,docID  2000
len(Topics)  386
iter:  4  ,docID  3000
len(Topics)  386
iter:  4  ,docID  4000
len(Topics)  382
iter:  4  ,docID  5000
len(Topics)  382
iter:  4  ,docID  6000
len(Topics)  382
iter:  4  ,docID  7000
len(Topics)  384
iter:  4  ,docID  8000
len(Topics)  384
iter:  4  ,docID  9000
len(Topics)  386
iter:  4  ,docID  10000
len(Topics)  388
iter:  4  ,docID  11000
len(Topics)  388
iter:  4  ,docID  12000
len(Topics)  389
iter:  4  ,docID  13000
len(Topics)  388
iter:  4  ,docID  14000
len(Topics)  389
iter:  4  ,docID  15000
len(Topics)  391
iter:  4  ,docID  16000
len(Topics)  394
iter:  4  ,docID  17000
len(Topics)  396
iter:  4  ,docID  18000
len(Topics)  397
iter:  4  ,docID  19000
len(Topics)  397
iter:  4  ,docID  20000
len(Topics)  397
iter:  4  ,docID  21000
len(Topics)  398
iter:  4  ,docID  22000
len(Topics)  398
iter:  4  ,docID  23000
len(Topics)  399
iter:  4  ,docID  24000
len(Topics)  399
iter:  4  ,docID  25000
l

In [18]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.8530557232491127, 0.8591912450637961, 0.8566488110641852, 0.8586708755451077, 0.8626523557338216, 0.8621394086867089, 0.8650346748479674, 0.8607455159529932, 0.8575112253597639, 0.8659697658558794])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [19]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([255, 295, 316, 313, 353, 358, 359, 388, 414, 408])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [14]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.8501170790654703, 0.8497837365672808, 0.8620309059889932, 0.8593399509327505, 0.8687509693321654, 0.8621786577877947, 0.8609508039846049, 0.8603121225412141, 0.8636778677384508, 0.8545787607757205])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [15]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([229, 277, 274, 291, 323, 356, 330, 359, 364, 344])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [3]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
#     if batch_i != 1:
#         start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  1895 topics:  92 truth:  32  NMI:  0.8430405745227187
[164.86401308362747]
start  0  end  3790 topics:  140 truth:  62  NMI:  0.831224553766329
[164.86401308362747, 481.74057526015366]
start  0  end  5685 topics:  140 truth:  79  NMI:  0.8412304483383486
[164.86401308362747, 481.74057526015366, 752.8474046528844]
start  0  end  7580 topics:  179 truth:  99  NMI:  0.8352214748247687
[164.86401308362747, 481.74057526015366, 752.8474046528844, 1169.9097144371506]
start  0  end  9475 topics:  191 truth:  122  NMI:  0.8368379181764768
[164.86401308362747, 481.74057526015366, 752.8474046528844, 1169.9097144371506, 1357.3181709603205]
start  0  end  11370 topics:  209 truth:  131  NMI:  0.8366874982339393
[164.86401308362747, 481.74057526015366, 752.8474046528844, 1169.9097144371506, 1357.3181709603205, 1738.1896780443303]
start  0  end  13265 topics:  214 truth:  144  NMI:  0.8477543588329287
[164.86401308362747, 481.74057526015366, 752.8474046528844, 1169.9097144371506, 1357.

In [4]:
print(topic_batch.keys())
print(topic_batch.values())

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([92, 140, 140, 179, 191, 209, 214, 234, 250, 249, 260, 273, 271, 276, 306, 312])
